# Starter notebook for NIH Chest Xray

## Copy data from GCS (if not so yet)

In [ ]:
# user-specific setting
PROJECT = 'mcsds-dlh'  # CHANGE: billing project name (since the dataset is user-to-pay)
DATA_FOLDER = '../data/'

In [ ]:
# Download images from GCS. Takes a few minutes.
# https://cloud.google.com/healthcare/docs/resources/public-datasets/nih-chest#gcp_data_access

#!gsutil -u {PROJECT} -m -q cp -r gs://gcs-public-data--healthcare-nih-chest-xray/png/*.png {DATA_FOLDER}

# Download addition labels
# https://pubs.rsna.org/doi/10.1148/radiol.2019191293

#!gsutil -u {PROJECT} -m -q cp -r gs://gcs-public-data--healthcare-nih-chest-xray-labels/* {DATA_FOLDER}

# Code starts here

In [ ]:
# import libraries
import pandas as pd
import numpy as np
import random
import os
import torch
from torch.utils.data import Dataset
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torch.nn.functional as F
from PIL import Image
from sklearn.preprocessing import MultiLabelBinarizer
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

# check if CUDA is available (GPU)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Device: {device}")

In [ ]:
# set seed
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
os.environ["PYTHONHASHSEED"] = str(seed)

In [ ]:
# explore the dataset
# load train test split
with open('train_val_list.txt') as f: 
    train_val_list = [x.strip() for x in f.readlines()]
with open('test_list.txt') as f:
    test_list = [x.strip() for x in f.readlines()]

# load labels
df_labels = pd.read_csv('Data_Entry_2017_v2020.csv')
print(f"Number of images: {len(df_labels)}")
# split the finding (disease) labels, to a list
df_labels['targets'] = df_labels['Finding Labels'].str.split("|", expand = False)
# look at available labels
labels = set([item for sublist in df_labels['targets'].tolist() for item in sublist])

print(f"Number of labels: {len(labels)}")
print(f"Labels: {labels}")

# one-hot encode labels to columns
mlb = MultiLabelBinarizer(sparse_output=True)

df_labels = df_labels.join(
            pd.DataFrame.sparse.from_spmatrix(
                mlb.fit_transform(df_labels.pop('targets')),
                index=df_labels.index,
                columns=mlb.classes_))
df_labels[list(labels)]=df_labels[list(labels)].sparse.to_dense()  # for easy .describe()

# show converted data
df_labels[['Finding Labels', *list(labels)]].head(10)

In [ ]:
df_labels.describe(include='all')

In [ ]:
# split into train_val and test sets
df_train_val = df_labels[df_labels['Image Index'].isin(train_val_list)]
df_test = df_labels[df_labels['Image Index'].isin(test_list)]

print(f"Number of train/val images: {len(df_train_val)}")
print(f"Number of test images: {len(df_test)}")

assert (len(df_train_val) + len(df_test)) == len(df_labels), "Total number of images does not equal to sum of train/val and test!"

In [ ]:
# read images
i=54756

print(df_labels['Image Index'][i])
path_to_img = os.path.join('../data', df_labels['Image Index'][i])

img=mpimg.imread(path_to_img)
print(f"Image size: {img.shape}")  # 2D
print(img.max(), img.min())  # grayscale [0.0, 1.0]
print(df_labels['Patient Age'][i])
print(df_labels['Patient Gender'][i])
print(df_labels['View Position'][i])  # only AP/PA, no lateral
# Plot an image
imgplot = plt.imshow(img, cmap='gray')

Take the label *Atelectasis* as pivot, let's build a classifier for it.

Settings:
1. Consider only PA view images.
2. Binary classification.

In [ ]:
disease = 'Atelectasis'

In [ ]:
# Label distribution
df_labels.describe(include='all')
df_labels[disease].hist()
print(f"Fraction of positive class: {len(df_labels[df_labels[disease]==1])/len(df_labels):.3f}")

In [ ]:
from sklearn.model_selection import train_test_split
df_train, df_val = train_test_split(df_train_val, test_size=0.2,random_state=seed)  # 20% val set, about same size as test

assert len(df_train) + len(df_val) == len(df_train_val)

# Prepare train/val and test data
def select_images(df):
    df = df[df['View Position']=='PA'].reset_index()
    return df

df_train_pa = select_images(df_train)
df_val_pa = select_images(df_val)
df_test_pa = select_images(df_test)

print(f"# train images: {df_train_pa.shape[0]}")
print(f"# val images: {df_val_pa.shape[0]}")
print(f"# test images: {df_test_pa.shape[0]}")

In [ ]:
df_train_pa.loc[90]

**Warning: The validation images serve as test set. Do NOT use them for model tuning.**
Use leave-out set/CV on training images for tuning instead.

Now we have the images and labels. We can train our model.

In [ ]:
# Loader

class NihDataset(Dataset):
    def __init__(self, dataframe, root_dir, label, transform=None):
        """
        label: column name of the label of interest, e.g. 'Pleural Effusion'.
        """
        self.dataframe = dataframe
        self.root_dir = root_dir
        self.transform = transform
        self.label = label
    
    def __len__(self):
        return len(self.dataframe)
    
    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.dataframe.loc[idx, 'Image Index'])
        image = Image.open(img_name).convert('L')  # via .getband(), some images are know to have 4 channels. Here we convert them to 1-channel grayscale.
        target = self.dataframe.loc[idx, disease]
            
        if self.transform:
            image = self.transform(image)
        
        return image, target
        

In [ ]:
# Data loaders to return batch of images
def load_data(dataframe, root_dir, label, transform=None, batch_size=32, shuffle=True, num_workers=4):
    '''
    Data Loader with batch loading and transform.
    '''
    image_data = NihDataset(dataframe, root_dir, label, transform=transform)
    pin = device=='cpu'
    loader = torch.utils.data.DataLoader(image_data, batch_size=batch_size, shuffle=shuffle, num_workers=num_workers, pin_memory=pin)
    return loader

data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop((112,112)),
        transforms.RandomHorizontalFlip(),  # data augmentation
        transforms.ToTensor(),
        #transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize((128,128)),
        transforms.CenterCrop(112),
        transforms.ToTensor(),
        #transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

Define our CNN architecture:

In [ ]:
# Helper function
def conv_output_volume(W, F, S, P):
    
    """
    Given the input volume size $W$, the kernel/filter size $F$, 
    the stride $S$, and the amount of zero padding $P$ used on the border, 
    calculate the output volume size.
    """
    return int((W - F + 2*P) / S) + 1

def maxpool_output_volume(W, F, S):
    
    """
    Given the input volume size $W$, the kernel/filter size $F$, 
    the stride $S$, and the amount of zero padding $P$ used on the border, 
    calculate the output volume size.
    """
    return int(np.ceil((W - F + 1) / S))

conv_layer1_size = conv_output_volume(W=112, F=5, S=1, P=0)
maxpool_layer1_size = maxpool_output_volume(W=conv_layer1_size, F=2, S=2)

conv_layer2_size = conv_output_volume(W=maxpool_layer1_size, F=5, S=1, P=0)
maxpool_layer2_size = maxpool_output_volume(W=conv_layer2_size, F=2, S=2)

print(conv_layer1_size, maxpool_layer1_size, conv_layer2_size, maxpool_layer2_size)

In [ ]:
# For now, just use a simple one from https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html, plus dropout

import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=3, kernel_size=5)  # stride=1, padding=0
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(3, 8, 5)
        self.fc1 = nn.Linear(8 * 25 * 25, 60)
        self.fc2 = nn.Linear(60, 20)
        self.fc3 = nn.Linear(20, 1)  # 2 classes
        self.dropout = nn.Dropout(p=0.2)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))  # size: (batch_size*)3channels*110*110
        x = self.pool(F.relu(self.conv2(x)))  # size: (batch_size*)8channels*53*53
        x = x.view(-1, 8 * 25 * 25)
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.dropout(F.relu(self.fc2(x)))
        x = torch.sigmoid(self.fc3(x))  # change to softmax if multiclass
        return x

model = Net().to(device)

In [ ]:
# Define loss function and optimizer

import torch.optim as optim

criterion = nn.BCELoss()  # change to CrossEntropyLoss if  multiclass
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
def train_model(model, train_loader, valid_loader, num_epochs=10, threshold=0.5):
    start_time = time.time()
    train_losses, val_losses = [], []
    for epoch in range(num_epochs):
        model.train()
        print(f"Epoch {epoch+1}")
        running_loss = 0.0
        for i, data in enumerate(train_loader, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.type(torch.FloatTensor).to(device)
            # zero the parameter gradients
            optimizer.zero_grad()
            # forward + backward + optimize
            outputs = model.forward(inputs)
            loss = criterion(outputs.squeeze(-1), labels)
            loss.backward()
            optimizer.step()

            # print statistics every 10 batches
            running_loss += loss.item()
            if i % 10 == 9:    # print every 10 mini-batches
                print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / (i+1)))
                print(f'Time elapsed: {(time.time()-start_time)/60.0:.1f} minutes.')
        train_losses.append(running_loss/len(train_data_loader))  # keep trace of train loss in each epoch
        
        # validate every epoch
        print("Validating...")
        val_loss, _, _, _ = eval_model(model, valid_loader, threshold=threshold)
        val_losses.append(val_loss/len(val_data_loader))  # keep trace of validation loss in each epoch
        print("Epoch: {}/{}.. ".format(epoch+1, num_epochs),
              "Training Loss: {:.3f}.. ".format(train_losses[-1]),
              "Validation Loss: {:.3f}.. ".format(val_losses[-1]))
        
        # save the model every epoch
        MODEL_PATH = f'../models/simple_net_{time.time()}_epoch{num_epochs}.pth'
        torch.save(model.state_dict(), MODEL_PATH)
        print("Model saved!")
        
    return model

def eval_model(model, loader, threshold=0.5):
    from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix, ConfusionMatrixDisplay, roc_curve
    # validate every epoch
    loss = 0.0
    # Turn off gradients for validation, saves memory and computations
    with torch.no_grad():
        model.eval()
        # empty tensors to hold results
        Y_prob, Y_true, Y_pred = [], [], []
        for inputs, labels in loader:
            probs = model(inputs.to(device)).squeeze(-1)  # prediction probability
            labels = labels.type(torch.FloatTensor).to(device)  # true labels
            predicted = probs > threshold  # predicted labels               
            # stack results
            Y_prob.append(probs)
            Y_true.append(labels)
            Y_pred.append(predicted)
            
            loss += criterion(probs, labels)

    # convert to numpy
    Y_prob = torch.cat(Y_prob).detach().cpu().numpy()
    Y_pred = torch.cat(Y_pred).detach().cpu().numpy()
    Y_true = torch.cat(Y_true).detach().cpu().numpy()

    # TODO: use desired metrics here
    print(f"ROC: {roc_auc_score(Y_true, Y_prob):.3f}")
    fpr, tpr, _ = roc_curve(Y_true, Y_prob)
    auc = roc_auc_score(Y_true, Y_prob)
    plt.plot(fpr,tpr,label="data 1, auc="+str(auc))
    plt.legend(loc=4)
    plt.show()
    
    print(classification_report(Y_true, Y_pred))
    cm = confusion_matrix(Y_true, Y_pred)  # confusion matrix
    disp = ConfusionMatrixDisplay(confusion_matrix=cm)
    disp.plot()
     
    return loss, Y_prob, Y_pred, Y_true

In [ ]:
import time

num_epochs = 10
batch_size = 2048

train_data_loader = load_data(df_train_pa, DATA_FOLDER, disease, transform=data_transforms['train'], shuffle=True, batch_size=batch_size)
val_data_loader = load_data(df_val_pa, DATA_FOLDER, disease, transform=data_transforms['test'], shuffle=False, batch_size=32)

print(f"Training start. Mode: {device}")
start_time = time.time()
model = train_model(model, train_data_loader, val_data_loader, num_epochs=num_epochs, threshold=0.10)
print(f'Finished Training. Total time: {time.time()-start_time} secs.')

In [ ]:
# Evaluate on test set
test_data_loader = load_data(df_test_pa, DATA_FOLDER, disease, transform=data_transforms['test'], shuffle=False, batch_size=32)
test_loss = eval_model(model, test_data_loader, threshold=0.4)